# Downstream Analysis: Healthy vs T2D Beta Cells

Analysis of pancreatic beta cell scRNA-seq data comparing healthy and T2D samples.

**Analyses:**
1. Beta-cell marker expression (INS, GCG, PPY, SST)
2. t-SNE visualization
3. Feature selection with logistic regression
4. T2D-associated gene analysis


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.stats import mannwhitneyu
import gzip
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

In [ ]:
# Load featureCounts output
counts_file = "results/gene_counts.txt"

counts_raw = pd.read_csv(counts_file, sep='\t', comment='#')

print(f"Raw data shape: {counts_raw.shape}")
print(f"\nColumn names:")
print(counts_raw.columns.tolist())
counts_raw.head()

In [ ]:
# Convert raw counts to RPKM

def counts_to_rpkm(counts_df):
    """Convert featureCounts output to RPKM"""
    rpkm_df = counts_df.copy()
    
    gene_lengths = rpkm_df['Length']
    
    count_cols = [col for col in rpkm_df.columns if col.endswith('.bam')]
    
    print(f"Converting {len(count_cols)} samples to RPKM...")
    
    for col in count_cols:
        total_counts = rpkm_df[col].sum()
        # RPKM = (counts * 1e9) / (gene_length * total_counts)
        rpkm_df[col] = (rpkm_df[col] * 1e9) / (gene_lengths * total_counts)
    
    return rpkm_df

rpkm = counts_to_rpkm(counts_raw)

print("\nRPKM conversion complete!")
print(f"RPKM data shape: {rpkm.shape}")
rpkm.head()

In [ ]:
# Transpose: rows = samples, columns = genes
rpkm_t = rpkm.drop(['Chr', 'Start', 'End', 'Strand', 'Length'], axis=1).set_index('Geneid').T

print(f"Transposed shape: {rpkm_t.shape}")
print(f"Samples: {list(rpkm_t.index)}")

In [ ]:
# Check the E-MTAB-5061.sdrf.txt file for metadata

# Load metadata to check sample conditions
metadata_file = "data/E-MTAB-5061.sdrf.txt"
metadata = pd.read_csv(metadata_file, sep='\t')

# Look for condition information
print("Metadata columns:")
print(metadata.columns.tolist())
print("\nFirst few rows:")
print(metadata.head())

# Find your samples in metadata - did manually for demonstration (5 samples)
sample_ids = ['ERR1630013', 'ERR1630018', 'ERR1630019', 'ERR1630033', 'ERR1630034']
sample_metadata = metadata[metadata['Comment[ENA_RUN]'].isin(sample_ids)]
print("\nYour samples metadata:")
print(sample_metadata[['Comment[ENA_RUN]', 'Characteristics[disease]']])

In [ ]:
# Assign conditions based on metadata
# Example - adjust based on actual metadata:

sample_conditions = {
    'ERR1630013.bam': 'healthy',  # Check metadata
    'ERR1630018.bam': 'healthy',
    'ERR1630019.bam': 'healthy',
    'ERR1630033.bam': 't2d',
    'ERR1630034.bam': 't2d'
}

rpkm_t['condition'] = rpkm_t.index.map(sample_conditions)
rpkm_t['sample'] = rpkm_t.index.str.replace('.bam', '')

print("Sample conditions:")
print(rpkm_t[['condition']].value_counts())

## 1. Beta-cell Marker Validation

In [ ]:
# Parse GTF to get gene_name -> gene_id mapping
def parse_gtf_for_markers(gtf_file, target_genes):
    """Extract specific genes from GTF"""
    gene_map = {}
    
    print(f"Parsing GTF for genes: {target_genes}")
    
    with gzip.open(gtf_file, 'rt') as f:
        for line in f:
            if line.startswith('#'):
                continue
            
            fields = line.strip().split('\t')
            if len(fields) < 9 or fields[2] != 'gene':
                continue
            
            # Parse attributes
            attrs = fields[8]
            gene_id = None
            gene_name = None
            
            for attr in attrs.split(';'):
                attr = attr.strip()
                if attr.startswith('gene_id'):
                    gene_id = attr.split('"')[1]
                elif attr.startswith('gene_name'):
                    gene_name = attr.split('"')[1]
            
            if gene_name in target_genes and gene_id:
                gene_map[gene_name] = gene_id
                
            # Stop early if we found all genes
            if len(gene_map) == len(target_genes):
                break
    
    return gene_map

# Beta-cell markers
markers = ['GCG', 'INS', 'PPY', 'SST']
gtf_file = "data/genome/Homo_sapiens.GRCh38.109.gtf.gz"

marker_ids = parse_gtf_for_markers(gtf_file, markers)
print("\nBeta-cell marker Ensembl IDs:")
for symbol, ens_id in marker_ids.items():
    print(f"  {symbol}: {ens_id}")

In [ ]:
# Extract marker expression
marker_expr = rpkm_t[[marker_ids[m] for m in markers if m in marker_ids]].copy()
marker_expr.columns = [m for m in markers if m in marker_ids]

# Log2 transform
log_expr = np.log2(marker_expr + 1)
log_expr['condition'] = rpkm_t['condition']

print("Beta-cell marker expression summary:")
print(log_expr.groupby('condition').mean())

In [ ]:
# Melt for plotting
long_df = log_expr.melt(id_vars='condition', var_name='Gene', value_name='Expression')

# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(data=long_df, x='Gene', y='Expression', hue='condition',
            palette={'healthy': '#4E79A7', 't2d': '#E15759'})
plt.ylabel('log2(RPKM + 1)')
plt.title('Beta-cell Markers: Healthy vs T2D')
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

In [ ]:
# Statistical testing
print("Mann-Whitney U test results:\n")
for gene in log_expr.columns:
    if gene == 'condition':
        continue
    
    healthy = log_expr[log_expr['condition'] == 'healthy'][gene].dropna()
    t2d = log_expr[log_expr['condition'] == 't2d'][gene].dropna()
    
    if len(healthy) > 0 and len(t2d) > 0:
        stat, p = mannwhitneyu(healthy, t2d, alternative='two-sided')
        sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
        print(f"{gene:5s}: U={stat:.2f}, p={p:.4f} {sig}")

## 2. t-SNE Visualization

In [ ]:
# Smaller sample size may not show clear separation amon healthy and t2d
# Prepare data for t-SNE
X = rpkm_t.drop(columns=['condition', 'sample'])
X_log = np.log2(X + 1)

print(f"Data shape: {X_log.shape}")
print(f"Samples: {len(X_log)}")

# PCA first (reduce dimensions)
n_components = min(3, len(X_log) - 1)  # Can't have more components than samples
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_log)

print(f"\nPCA variance explained: {pca.explained_variance_ratio_.sum():.2%}")

# t-SNE
perplexity = min(2, len(X_log) - 1)  # Perplexity must be less than n_samples
tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
X_tsne = tsne.fit_transform(X_pca)

print("t-SNE complete!")

In [ ]:
# Plot t-SNE
plt.figure(figsize=(8, 6))

colors = {'healthy': '#4E79A7', 't2d': '#E15759'}
markers_style = {'healthy': 'o', 't2d': 's'}

for cond in rpkm_t['condition'].unique():
    mask = rpkm_t['condition'] == cond
    plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1],
                label=cond, c=colors[cond], marker=markers_style[cond],
                s=150, alpha=0.8, edgecolors='black', linewidth=1.5)

plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title(f't-SNE Visualization (n={len(X_log)} samples)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Feature Selection

In [ ]:
# For small samples, ML results will have high variance
# Prepare labels
y = rpkm_t['condition']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"Classes: {le.classes_}")
print(f"Distribution: {dict(zip(le.classes_, np.bincount(y_encoded)))}")

clf = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, C=0.1, random_state=42)
clf.fit(X_log, y_encoded)

print(f"\nModel trained on {len(X_log)} samples")
print("Note: No train/test split due to small sample size")

In [ ]:
# Extract top predictive genes
feature_imp = pd.Series(clf.coef_[0], index=X_log.columns)
top_genes = feature_imp.abs().sort_values(ascending=False).head(20)

# Build reverse mapping (Ensembl ID -> gene symbol)
# Parse full GTF for all genes
def get_full_gene_mapping(gtf_file, gene_ids):
    """Get gene symbols for list of Ensembl IDs"""
    id_to_symbol = {}
    
    with gzip.open(gtf_file, 'rt') as f:
        for line in f:
            if line.startswith('#') or '\tgene\t' not in line:
                continue
            
            attrs = line.split('\t')[8]
            gene_id = None
            gene_name = None
            
            for attr in attrs.split(';'):
                attr = attr.strip()
                if 'gene_id' in attr:
                    gene_id = attr.split('"')[1]
                elif 'gene_name' in attr:
                    gene_name = attr.split('"')[1]
            
            if gene_id in gene_ids and gene_name:
                id_to_symbol[gene_id] = gene_name
    
    return id_to_symbol

print("Mapping top genes to symbols...")
id_to_symbol = get_full_gene_mapping(gtf_file, set(top_genes.index))

# Name the genes
top_genes_named = top_genes.rename(index=lambda x: id_to_symbol.get(x, x))

print("\nTop 20 predictive genes:")
print(top_genes_named)

In [ ]:
# Plot top genes
plt.figure(figsize=(10, 8))
top_genes_named.sort_values().plot(kind='barh', color='steelblue')
plt.xlabel('|Coefficient| (L1 Logistic Regression)')
plt.ylabel('Gene')
plt.title('Top 20 Predictive Genes (Healthy vs T2D)')
plt.tight_layout()
plt.show()

In [ ]:
# Check for known T2D-associated genes
t2d_genes = ['TCF7L2', 'CDKAL1', 'KCNJ11', 'PPARG', 'SLC30A8', 'GCK', 
             'HNF1A', 'HNF4A', 'INS', 'GCG', 'IRS1', 'GLIS3']

top_symbols = set(top_genes_named.index)
overlap = top_symbols & set(t2d_genes)

print(f"Known T2D genes in top 20 predictors: {len(overlap)}")
if overlap:
    print(f"Genes: {', '.join(sorted(overlap))}")
else:
    print("No known T2D genes in top 20 (likely due to small sample size)")